# Sparse polynomial optimization using correlative sparsity

## A motivating toy example

One approach for exploiting sparsity in polynomial optimization is based on <i>correlative sparsity</i> and proposed by Waki, Kim, Kojima and Muramatsu. We motivate this approach by considering the following problem,

\begin{array}{ll}
\text{minimize}   & x_1 - x_2 + x_3 - (x_1^3 + x_2^3 + x_3^3)\\
\text{subject to} & -1 \leq x_1 x_2 \leq 1\\
                  & -1 \leq x_2 x_3 \leq 1\\
                  & x_1^2 \leq 1\\
                  & x_2^2 \leq 1\\
                  & x_3^2 \leq 1\\
\end{array}

We first setup and solve the standard second-order moment relaxation

In [133]:
include("..\\src\\Polyopt.jl")

Main.Polyopt

In [134]:
x1,x2,x3 = Polyopt.variables(["x1","x2","x3"]);

In [135]:
f = x1-x2+x3 - (x1^3+x2^3+x3^3);

In [136]:
 g = [-1-x1*x2, 1-x1*x2,
      -1-x2*x3, 1-x2*x3,
       1-x1^2,
       1-x2^2,
       1-x3^2 ];

In [137]:
prob = Polyopt.momentprob(2, f, g);

In [138]:
X, Z, t, y, solsta = Polyopt.solve_mosek(prob);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 35              
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 8               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.02            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.05    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                  

We inspect the solution

In [139]:
[ prob.basis y ]

35×2 Array{Main.Polyopt.Poly{Float64},2}:
 1.0         1.0
 x3          -1.0
 x3^2        1.0
 x3^3        -1.0
 x3^4        1.0
 x2          1.0
 x2*x3       -1.0
 x2*x3^2     1.0
 x2*x3^3     -1.0
 x2^2        1.0
 x2^2*x3     -1.0
 x2^2*x3^2   1.0
 x2^3        1.0
 ⋮           
 x1*x2^2*x3  1.0
 x1*x2^3     -1.0
 x1^2        1.0
 x1^2*x3     -1.0
 x1^2*x3^2   1.0
 x1^2*x2     1.0
 x1^2*x2*x3  -1.0
 x1^2*x2^2   1.0
 x1^3        -1.0
 x1^3*x3     1.0
 x1^3*x2     -1.0
 x1^4        1.0

which is feasible

In [140]:
xo = Polyopt.vectorize([x1,x2,x3],4)*y

3-element Array{Float64,1}:
 -0.9999999999976845
  0.9999999999978969
 -0.9999999999976845

In [141]:
[ Polyopt.evalpoly(gi, xo) for gi=g ]

7-element Array{Float64,1}:
 -4.4185766157056605e-12
  1.9999999999955813
 -4.4185766157056605e-12
  1.9999999999955813
  4.630962280316453e-12
  4.206190951094868e-12
  4.630962280316453e-12

so we have found an optimal solution.

## Correlative sparsity

The correlative sparsity pattern $C$ is a symmetric matrix with the number variables as dimension.
An element $C_{ij}$ is nonzero if either
* $i=j$, i.e., we include the diagonal.
* $x_i$ and $x_j$ appear in the same term of the objective function.
* $x_i$ and $x_j$ both appear in a constraint $g(x)\geq 0$ or $h(x)=0$.

For the simple example, let us recall

In [142]:
f

x3-x3^3-x2-x2^3+x1-x1^3

In [143]:
g

7-element Array{Main.Polyopt.Poly{Int64},1}:
 -1-x1*x2
 1-x1*x2
 -1-x2*x3
 1-x2*x3
 1-x1^2
 1-x2^2
 1-x3^2

Let us compute the correlative sparsity pattern

In [144]:
C = Polyopt.correlative_sparsity(f, g);

In [145]:
Matrix(C)

3×3 Array{Int64,2}:
 1  1  0
 1  1  1
 0  1  1

This is a <i>chordal</i> matrix with 2 cliques,

In [146]:
K = Array{Int,1}[ [1,2], [2,3] ]

2-element Array{Array{Int64,1},1}:
 [1, 2]
 [2, 3]

## A weaker chordal moment relaxation formed over the cliques 

In the standard moment relaxation we consider a monomial basis vector

In [147]:
u = Polyopt.monomials(2, [x1,x2,x3])

10-element Array{Main.Polyopt.Poly{Int64},1}:
 1
 x3
 x3^2
 x2
 x2*x3
 x2^2
 x1
 x1*x3
 x1*x2
 x1^2

and the second-order moment matrix $M_2(y)\succeq 0$ corresponds to

In [148]:
u*u'

10×10 Array{Main.Polyopt.Poly{Int64},2}:
 1      x3        x3^2        x2        …  x1*x3       x1*x2       x1^2
 x3     x3^2      x3^3        x2*x3        x1*x3^2     x1*x2*x3    x1^2*x3
 x3^2   x3^3      x3^4        x2*x3^2      x1*x3^3     x1*x2*x3^2  x1^2*x3^2
 x2     x2*x3     x2*x3^2     x2^2         x1*x2*x3    x1*x2^2     x1^2*x2
 x2*x3  x2*x3^2   x2*x3^3     x2^2*x3      x1*x2*x3^2  x1*x2^2*x3  x1^2*x2*x3
 x2^2   x2^2*x3   x2^2*x3^2   x2^3      …  x1*x2^2*x3  x1*x2^3     x1^2*x2^2
 x1     x1*x3     x1*x3^2     x1*x2        x1^2*x3     x1^2*x2     x1^3
 x1*x3  x1*x3^2   x1*x3^3     x1*x2*x3     x1^2*x3^2   x1^2*x2*x3  x1^3*x3
 x1*x2  x1*x2*x3  x1*x2*x3^2  x1*x2^2      x1^2*x2*x3  x1^2*x2^2   x1^3*x2
 x1^2   x1^2*x3   x1^2*x3^2   x1^2*x2      x1^3*x3     x1^3*x2     x1^4

In the chordal relaxation we consider basis vectors formed over the cliques

In [149]:
uk = [ Polyopt.monomials(2, [x1,x2,x3][ki]) for ki=K ]

2-element Array{Array{Main.Polyopt.Poly{Int64},1},1}:
 [1, x2, x2^2, x1, x1*x2, x1^2]
 [1, x3, x3^2, x2, x2*x3, x2^2]

and the moment constraint $M_2(y)\succeq 0$ is replaced by two smaller constraints $M_2(y, K_i)\succeq 0$ corresponding to

In [150]:
uk[1]*uk[1]'

6×6 Array{Main.Polyopt.Poly{Int64},2}:
 1      x2       x2^2       x1       x1*x2      x1^2
 x2     x2^2     x2^3       x1*x2    x1*x2^2    x1^2*x2
 x2^2   x2^3     x2^4       x1*x2^2  x1*x2^3    x1^2*x2^2
 x1     x1*x2    x1*x2^2    x1^2     x1^2*x2    x1^3
 x1*x2  x1*x2^2  x1*x2^3    x1^2*x2  x1^2*x2^2  x1^3*x2
 x1^2   x1^2*x2  x1^2*x2^2  x1^3     x1^3*x2    x1^4

In [151]:
uk[2]*uk[2]'

6×6 Array{Main.Polyopt.Poly{Int64},2}:
 1      x3       x3^2       x2       x2*x3      x2^2
 x3     x3^2     x3^3       x2*x3    x2*x3^2    x2^2*x3
 x3^2   x3^3     x3^4       x2*x3^2  x2*x3^3    x2^2*x3^2
 x2     x2*x3    x2*x3^2    x2^2     x2^2*x3    x2^3
 x2*x3  x2*x3^2  x2*x3^3    x2^2*x3  x2^2*x3^2  x2^3*x3
 x2^2   x2^2*x3  x2^2*x3^2  x2^3     x2^3*x3    x2^4

Similarly we consider the <i>localization matrices</i> for $g_i(x)\geq 0$. For the standard second-order moment relexation we have

In [152]:
v = Polyopt.monomials(1, [x1,x2,x3])

4-element Array{Main.Polyopt.Poly{Int64},1}:
 1
 x3
 x2
 x1

In [153]:
g[1]

-1-x1*x2

and the localization matrix for $g_1(x) \geq 0$ is $M_1(g_1 y)\succeq 0$ corresponding to

In [154]:
g[1]*v*v'

4×4 Array{Main.Polyopt.Poly{Int64},2}:
 -1-x1*x2      -x3-x1*x2*x3       -x2-x1*x2^2        -x1-x1^2*x2
 -x3-x1*x2*x3  -x3^2-x1*x2*x3^2   -x2*x3-x1*x2^2*x3  -x1*x3-x1^2*x2*x3
 -x2-x1*x2^2   -x2*x3-x1*x2^2*x3  -x2^2-x1*x2^3      -x1*x2-x1^2*x2^2
 -x1-x1^2*x2   -x1*x3-x1^2*x2*x3  -x1*x2-x1^2*x2^2   -x1^2-x1^3*x2

In the chordal relaxation we have

In [155]:
vk = [ Polyopt.monomials(1, [x1,x2,x3][ki]) for ki=K ]

2-element Array{Array{Main.Polyopt.Poly{Int64},1},1}:
 [1, x2, x1]
 [1, x3, x2]

and we replace $M_1(g_1 y)\succeq 0$ by $M_1(g_1 y, K_1)\succeq 0$ corresponding to

In [156]:
g[1]*vk[1]*vk[1]'

3×3 Array{Main.Polyopt.Poly{Int64},2}:
 -1-x1*x2     -x2-x1*x2^2       -x1-x1^2*x2
 -x2-x1*x2^2  -x2^2-x1*x2^3     -x1*x2-x1^2*x2^2
 -x1-x1^2*x2  -x1*x2-x1^2*x2^2  -x1^2-x1^3*x2

Similarly we replace $M_1(g_2y)$ by

In [157]:
g[2]*vk[1]*vk[1]'

3×3 Array{Main.Polyopt.Poly{Int64},2}:
 1-x1*x2     x2-x1*x2^2       x1-x1^2*x2
 x2-x1*x2^2  x2^2-x1*x2^3     x1*x2-x1^2*x2^2
 x1-x1^2*x2  x1*x2-x1^2*x2^2  x1^2-x1^3*x2

and so on. The function $g_6(x)=1-x_2^2$ belongs to both clique 1 and 2, so we replace $M_1(g_6 y)\succeq 0$ by two smaller inequalities

In [158]:
g[6]*vk[1]*vk[1]'

3×3 Array{Main.Polyopt.Poly{Int64},2}:
 1-x2^2      x2-x2^3        x1-x1*x2^2
 x2-x2^3     x2^2-x2^4      x1*x2-x1*x2^3
 x1-x1*x2^2  x1*x2-x1*x2^3  x1^2-x1^2*x2^2

and

In [159]:
g[6]*vk[2]*vk[2]'

3×3 Array{Main.Polyopt.Poly{Int64},2}:
 1-x2^2      x3-x2^2*x3      x2-x2^3
 x3-x2^2*x3  x3^2-x2^2*x3^2  x2*x3-x2^3*x3
 x2-x2^3     x2*x3-x2^3*x3   x2^2-x2^4

## Solving the chordal moment relaxation

We form the chordal moment relaxation as

In [160]:
probc = Polyopt.momentprob_chordal(2, K, f, g);

and we solve it as

In [161]:
Xc, Zc, tc, yc, solstac = Polyopt.solve_mosek(probc);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 25              
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 10              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                  

In [162]:
Matrix([ probc.basis yc ])

35×2 Array{Main.Polyopt.Poly{Float64},2}:
 1.0         1.0
 x3          -1.0
 x3^2        1.0
 x3^3        -1.0
 x3^4        1.0
 x2          1.0
 x2*x3       -1.0
 x2*x3^2     1.0
 x2*x3^3     -1.0
 x2^2        1.0
 x2^2*x3     -1.0
 x2^2*x3^2   1.0
 x2^3        1.0
 ⋮           
 x1*x2^2*x3  0.0
 x1*x2^3     -1.0
 x1^2        1.0
 x1^2*x3     0.0
 x1^2*x3^2   0.0
 x1^2*x2     1.0
 x1^2*x2*x3  0.0
 x1^2*x2^2   1.0
 x1^3        -1.0
 x1^3*x3     0.0
 x1^3*x2     -1.0
 x1^4        1.0

In this case we also recover the optimal value at the second order relaxation. In general this is not always the case, except when all polynomials are quadratic, but under mild assumptions the hierarchy of chordal moment relaxations converges to the optimal solution (just as the regular moment relaxations).

## Duality and sums-of-squares certificates

The sums-of-squares certificate is also evaluted over the cliques, in this case we have

In [163]:
using LinearAlgebra
r = dot(uk[1], Xc[1]*uk[1]) + dot(uk[2], Xc[2]*uk[2]) + 
        g[1]*dot(vk[1], Xc[3]*vk[1]) +
        g[2]*dot(vk[1], Xc[4]*vk[1]) +
        g[3]*dot(vk[2], Xc[5]*vk[2]) +
        g[4]*dot(vk[2], Xc[6]*vk[2]) +
        g[5]*dot(vk[1], Xc[7]*vk[1]) +
        g[6]*(dot(vk[1], Xc[8]*vk[1]) + dot(vk[2], Xc[9]*vk[2])) +
        g[7]*dot(vk[2], Xc[10]*vk[2]);

In [164]:
truncate( f - tc - r, 1e-6 )

0.0

## Chordal embedding

When the correlative sparsity pattern is non-chordal it is difficult to find the cliques (it is NP-hard in general). In that case we embed the correlative sparsity pattern in a larger chordal graph for which we can easily detect the cliques. The chordal embedding procedure essentially corresponds to a symbolic Cholesky factorization of the correlative sparsity pattern, and the amount of generated fill-in depends on the quality of a symmetric matrix reordering (performed by <code>CHOLMOD</code>). 

To demonstrate this procedure we consider a problem from http://gamsworld.org/global/globallib/ex5_4_2.htm, where we have rescaled the problem such that $x_i\leq 1$ to make the relaxations easier to solve. 

In [165]:
x1,x2,x3,x4,x5,x6,x7,x8 = Polyopt.variables(["x1","x2","x3","x4","x5","x6","x7","x8"]);

In [166]:
f = x1 + x2 + x3;

In [167]:
g = [ 400/1000 - (x4 + x6),
      300/1000 - (-x4 + x5 + x7),
      100/1000 - (-x5 + x8),
      .083333 - (0.01*x1 - 10*x1*x6 + 0.83333*x4),      
      -(x2*x4 - x2*x7 - 0.125*x4 + 0.125*x5),
      -0.125 - (x3*x5 - x3*x8 - 0.25*x5),
      x1-100/10000, 1-x1, 
      x2-1000/10000, 1-x2, 
      x3-1000/10000, 1-x3, 
      x4-10/1000, 1-x4, 
      x5-10/1000, 1-x5, 
      x6-10/1000, 1-x6, 
      x7-10/1000, 1-x7, 
      x8-10/1000, 1-x8 ];

For reference, let us first solve the third-order standard moment relaxation.

In [168]:
prob = Polyopt.momentprob(3, f, g);

In [169]:
X, Z, t, y, solsta = Polyopt.solve_mosek(prob);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3003            
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 23              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                  

and the extracted solution 

In [170]:
[ prob.basis y ]

3003×2 Array{Main.Polyopt.Poly{Float64},2}:
 1.0         1.0
 x8          0.380589
 x8^2        0.144848
 x8^3        0.0551275
 x8^4        0.0209809
 x8^5        0.0079851
 x8^6        2.23737
 x7          0.284471
 x7*x8       0.108267
 x7*x8^2     0.041205
 x7*x8^3     0.0156822
 x7*x8^4     0.00596846
 x7*x8^5     -0.000457155
 ⋮           
 x1^4*x2*x4  -0.213214
 x1^4*x2*x3  0.00180014
 x1^4*x2^2   0.766096
 x1^5        1.1422e-5
 x1^5*x8     0.00618647
 x1^5*x7     0.00911014
 x1^5*x6     0.132243
 x1^5*x5     0.0175621
 x1^5*x4     0.0169488
 x1^5*x3     0.011129
 x1^5*x2     0.0139941
 x1^6        2.12709

is feasible

In [171]:
xo = Polyopt.vectorize([x1,x2,x3,x4,x5,x6,x7,x8],6)*y

8-element Array{Float64,1}:
 0.10269480182448586
 0.10000000000001597
 0.5485278018454621
 0.2650599826712479
 0.2805888792618106
 0.13494001732873756
 0.2844711034094327
 0.3805888792618064

In [172]:
[ Polyopt.evalpoly(gi, xo) for gi=g ]

22-element Array{Float64,1}:
  1.454392162258955e-14
  4.6074255521944e-15
  4.163336342344337e-15
 -1.970645868709653e-15
 -1.5439038936193583e-15
 -3.4139358007223564e-15
  0.09269480182448586
  0.8973051981755141
  1.5959455978986625e-14
  0.899999999999984
  0.44852780184546215
  0.4514721981545379
  0.2550599826712479
  0.7349400173287521
  0.2705888792618106
  0.7194111207381895
  0.12494001732873757
  0.8650599826712624
  0.2744711034094327
  0.7155288965905673
  0.3705888792618064
  0.6194111207381936

Now, let us form the chordal moment relaxation using a chordal embedding,

In [173]:
cholesky

cholesky (generic function with 9 methods)

In [174]:
probc = Polyopt.momentprob_chordalembedding(3, f, g);


ErrorException: type Array has no field ordering

In [175]:
Xc, Zc, tc, yc, solstac = Polyopt.solve_mosek(probc);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 25              
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 10              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                  

In [176]:
Matrix([ prob.basis y yc ])

DimensionMismatch: DimensionMismatch("")